In [ ]:
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Tuple
import pandas as pd
from collections import defaultdict as _dd
import hashlib, math, bisect
import random as _rnd

# Constants 

In [ ]:
SLOTS_PER_EPOCH   = 24000
SLOT_DURATION_SEC = 30
EEP_FRACTION      = 1/3
EEP_SLOT          = int(SLOTS_PER_EPOCH * EEP_FRACTION)

PENALTY_FACTOR        = 2.0
RECOVERY_FACTOR       = 1.0
TOLERATED_BAD_RATIO   = 0.02
REWARD_PER_BLOCK      = 0.0001  # coinbase in TKG per blocco valido
AUTO_COMPOUND         = True
RNG_SEED              = 42
MIN_BET_AMOUNT        = 50.0
TOKEN_SCALE_INT       = 10**18
W_SKIPPED             = 1.0
W_INVALID             = 2.0
W_DELAYED             = 1.5

In [ ]:
FEE_SCHEDULE_TKR: Dict[str, float] = {
    "PAY":                0.0001,
    "STAKE":              0.0010,
    "STAKE_UNDO":         0.0010,
    "REGISTER_MAIN":      1.0,
    "REGISTER_OVERFLOW":  0.1,
    "DEREGISTER_MAIN":    0.5,
    "DEREGISTER_OVERFLOW":0.05,
    "ASSIGN_OVERFLOW":    0.05,
    "UPLOAD_CONTRACT":    10.0,
    "CALL_FUNCTION":      0.01,
}

# Classes

In [ ]:
@dataclass
class Wallet:
    address: str
    green: float = 0.0   # TKG
    red: float   = 0.0   # TKR

    def get_balance(self) -> Dict[str, float]:
        return {"green": self.green, "red": self.red}

    def deposit(self, *, green: float = 0.0, red: float = 0.0) -> None:
        if green > 0: self.green += green
        if red   > 0: self.red   += red

    def transfer(self, other: "Wallet", *, green_amount: float = 0.0, red_amount: float = 0.0) -> bool:
        if green_amount < 0 or red_amount < 0: return False
        if green_amount > self.green or red_amount > self.red: return False
        if green_amount:
            self.green -= green_amount; other.green += green_amount
        if red_amount:
            self.red   -= red_amount;   other.red   += red_amount
        return (green_amount > 0) or (red_amount > 0)

In [ ]:
@dataclass
class Wallet:
    address: str
    green: float = 0.0   # TKG
    red: float   = 0.0   # TKR

    def get_balance(self) -> Dict[str, float]:
        return {"green": self.green, "red": self.red}

    def deposit(self, *, green: float = 0.0, red: float = 0.0) -> None:
        if green > 0: self.green += green
        if red   > 0: self.red   += red

    def transfer(self, other: "Wallet", *, green_amount: float = 0.0, red_amount: float = 0.0) -> bool:
        if green_amount < 0 or red_amount < 0: return False
        if green_amount > self.green or red_amount > self.red: return False
        if green_amount:
            self.green -= green_amount; other.green += green_amount
        if red_amount:
            self.red   -= red_amount;   other.red   += red_amount
        return (green_amount > 0) or (red_amount > 0)

In [ ]:
@dataclass
class MainAddress(Wallet):
    delegates: Dict[str, float] = field(default_factory=dict)
    active_stake: float = 0.0
    overflows: Dict[str, "OverflowNode"] = field(default_factory=dict)

    def register_overflow(self, overflow: "OverflowNode") -> None:
        overflow.main_id = self.address
        self.overflows[overflow.address] = overflow

    def get_overflow_list(self) -> list:
        return list(self.overflows.values())

    def bet(self, from_address: str, amount: float) -> bool:
        if amount < MIN_BET_AMOUNT: return False
        self.delegates[from_address] = self.delegates.get(from_address, 0.0) + amount
        return True

    def self_bet(self, amount: float) -> bool:
        if amount < MIN_BET_AMOUNT or amount > self.green: return False
        self.green -= amount
        return self.bet("_self_", amount)

    def self_unbet(self, amount: Optional[float] = None) -> float:
        cur = self.delegates.get("_self_", 0.0)
        if cur <= 0: return 0.0
        take = cur if amount is None else max(0.0, min(amount, cur))
        if take <= 0: return 0.0
        residual = cur - take
        if residual > 0: self.delegates["_self_"] = residual
        else:            self.delegates.pop("_self_", None)
        self.green += take
        return take

    def total_delegated_now(self) -> float:
        return sum(self.delegates.values())

    def snapshot_at_eep(self) -> None:
        self.active_stake = self.total_delegated_now()

    def credit_reward(self, amount_green: float) -> None:
        """Distribuisce coinbase in TKG (green) pro-rata nelle deleghe (compounding)."""
        if amount_green <= 0: return
        total = self.total_delegated_now()
        if total <= 0:
            self.green += amount_green
            return
        for addr, stake_amt in list(self.delegates.items()):
            share = amount_green * (stake_amt / total)
            self.delegates[addr] = stake_amt + share

In [ ]:
@dataclass
class OverflowNode(Wallet):
    main_id: str = ""
    base_stake_share: float = 1.0

    # comportamento rete
    p_skip: float = 0.00
    p_invalid: float = 0.00               # “errore protocollare” puro
    p_delayed: float = 0.00

    # comportamento sulle transazioni
    p_bad_tx: float = 0.0                  # prova a includere tx invalide/sottopagate

    # stato
    penalty_score: float = 0.0
    effective_stake: float = 0.0
    assigned_slots: int = 0
    penalty_slots_residual: int = 0

    def behavior_outcome(self, rng) -> str:
        x = rng.random()
        if x < self.p_skip:    return "skipped"
        x -= self.p_skip
        if x < self.p_invalid: return "invalid"
        x -= self.p_invalid
        if x < self.p_delayed: return "delayed"
        return "ok"

    def slash_green(self, amount: float) -> float:
        if amount <= 0: return 0.0
        taken = min(self.green, amount)
        self.green -= taken
        return taken

In [ ]:
@dataclass
class OverflowNode(Wallet):
    main_id: str = ""
    base_stake_share: float = 1.0

    p_skip: float = 0.00
    p_invalid: float = 0.00
    p_delayed: float = 0.00

    p_bad_tx: float = 0.0

    penalty_score: float = 0.0
    effective_stake: float = 0.0
    assigned_slots: int = 0
    penalty_slots_residual: int = 0

    def behavior_outcome(self, rng) -> str:
        x = rng.random()
        if x < self.p_skip:    return "skipped"
        x -= self.p_skip
        if x < self.p_invalid: return "invalid"
        x -= self.p_invalid
        if x < self.p_delayed: return "delayed"
        return "ok"

    def slash_green(self, amount: float) -> float:
        if amount <= 0: return 0.0
        taken = min(self.green, amount)
        self.green -= taken
        return taken

In [ ]:
class TakamakaSim:
    """
    Simulatore “slot-based” con schedule deterministica, penalty residual e
    reward = coinbase (TKG, green) + fee (TKR, red) al Main del produttore di blocchi validi.
    """

    def __init__(self,
                 *,
                 slots_per_epoch: int = SLOTS_PER_EPOCH,
                 slot_duration_sec: int = SLOT_DURATION_SEC,
                 eep_fraction: float = EEP_FRACTION,
                 penalty_factor: float = PENALTY_FACTOR,
                 recovery_factor: float = RECOVERY_FACTOR,
                 tolerated_bad_ratio: float = TOLERATED_BAD_RATIO,
                 reward_per_block: float = REWARD_PER_BLOCK,
                 seed: int = RNG_SEED,
                 slash_per_invalid: float = 0.0,
                 penalty_alpha: float = 1e-3,
                 w_skipped: float = W_SKIPPED,
                 w_invalid: float = W_INVALID,
                 w_delayed: float = W_DELAYED,
                 txs_per_slot_mean: int = 50,
                 max_txs_per_block: int = 200):
        self.cfg = {
            "slots_per_epoch": int(slots_per_epoch),
            "slot_duration_sec": int(slot_duration_sec),
            "eep_slot": int(slots_per_epoch * eep_fraction),
            "penalty_factor": float(penalty_factor),
            "recovery_factor": float(recovery_factor),
            "tolerated_bad_ratio": float(tolerated_bad_ratio),
            "reward_per_block": float(reward_per_block),
            "seed": int(seed),
            "slash_per_invalid": float(slash_per_invalid),
            "stake_scale_int": int(TOKEN_SCALE_INT),
            "penalty_alpha": float(penalty_alpha),
            "w_skipped": float(w_skipped),
            "w_invalid": float(w_invalid),
            "w_delayed": float(w_delayed),
            "txs_per_slot_mean": int(txs_per_slot_mean),
            "max_txs_per_block": int(max_txs_per_block),
        }
        self.epoch = 0

        self.stakeholders: Dict[str, Stakeholder] = {}
        self.mains: Dict[str, MainAddress] = {}
        self.overflows: Dict[str, OverflowNode] = {}

        self.pending_ops: List[Dict] = []
        self.history: List[Dict] = []

        self.weights_for_epoch: Dict[str, float] = {}
        self.weights_next_epoch: Dict[str, float] = {}

        self.current_schedule: List[str] = []
        self.next_schedule: List[str] = []

        self.current_seed: bytes = b""
        self.next_seed: bytes = b""

        self._slot_outcomes_current: List[str] = []
        self._bootstrap_done = False
        self._initialized_weights = False

        # mempool
        self.mempool: List[Transaction] = []

    # ---------------------------
    # Registrazione / indexing
    # ---------------------------
    def register_stakeholder(self, st: Stakeholder):
        self.stakeholders[st.address] = st

    def register_main(self, m: MainAddress):
        self.mains[m.address] = m

    def index_overflow(self, node: OverflowNode):
        if node.main_id not in self.mains:
            raise ValueError("main_id non registrato")
        self.overflows[node.address] = node

    # ---------------------------
    # Operazioni differite (EEP window)
    # ---------------------------
    def queue_bet(self, from_addr: str, to_main_id: str, amount: float, when_slot: int):
        if to_main_id not in self.mains:
            raise ValueError(f"Main non registrato: {to_main_id}")
        if from_addr not in self.stakeholders and from_addr != "_self_":
            raise ValueError(f"Stakeholder non registrato: {from_addr}")
        if amount < MIN_BET_AMOUNT:
            raise ValueError(f"Importo minimo di puntata è {MIN_BET_AMOUNT}, got {amount}")
        self.pending_ops.append({
            "kind": "STAKE",
            "from": from_addr,
            "to": to_main_id,
            "amount": float(amount),
            "epoch": self.epoch,
            "slot": int(when_slot),
        })

    def queue_stake_undo(self, from_addr: str, when_slot: int):
        if from_addr not in self.stakeholders:
            raise ValueError(f"Stakeholder non registrato: {from_addr}")
        self.pending_ops.append({
            "kind": "STAKE_UNDO",
            "from": from_addr,
            "epoch": self.epoch,
            "slot": int(when_slot),
        })

    def queue_self_unbet(self, main_id: str, when_slot: int, amount: Optional[float] = None):
        if main_id not in self.mains:
            raise ValueError(f"Main non registrato: {main_id}")
        self.pending_ops.append({
            "kind": "SELF_UNBET",
            "to": main_id,
            "amount": None if amount is None else float(amount),
            "epoch": self.epoch,
            "slot": int(when_slot),
        })

    def _apply_ops_at_current_eep(self):
        eep = self.cfg["eep_slot"]
        eligible, keep = [], []
        for op in self.pending_ops:
            if (op["epoch"] == self.epoch and op["slot"] <= eep) or \
               (op["epoch"] == self.epoch - 1 and op["slot"] > eep):
                eligible.append(op)
            else:
                keep.append(op)

        def op_key(op):
            return (0 if op["kind"] in ("STAKE_UNDO", "SELF_UNBET") else 1,
                    op.get("epoch", -1), op.get("slot", -1), op.get("from", ""), op.get("to", ""))

        eligible.sort(key=op_key)

        for op in eligible:
            kind = op["kind"]
            if kind == "STAKE_UNDO":
                deleg_addr = op["from"]
                st = self.stakeholders.get(deleg_addr)
                if not st: continue
                for m in self.mains.values():
                    cur = m.delegates.get(deleg_addr, 0.0)
                    if cur > 0:
                        st.green += cur
                        m.delegates.pop(deleg_addr, None)

            elif kind == "STAKE":
                to_main = self.mains.get(op["to"])
                if not to_main: continue
                amount = float(op["amount"])
                if amount < MIN_BET_AMOUNT: continue
                from_addr = op["from"]
                if from_addr == "_self_":
                    if amount <= to_main.green:
                        to_main.green -= amount
                        to_main.bet("_self_", amount)
                else:
                    st = self.stakeholders.get(from_addr)
                    if st and amount <= st.green:
                        st.green -= amount
                        to_main.bet(from_addr, amount)

            elif kind == "SELF_UNBET":
                to_main = self.mains.get(op["to"])
                if to_main:
                    amt = op.get("amount", None)
                    to_main.self_unbet(amt)

        self.pending_ops = keep

    def _snapshot_all_mains_at_eep(self):
        for m in self.mains.values():
            m.snapshot_at_eep()

    # ---------------------------
    # Helpers hashing/seed
    # ---------------------------
    @staticmethod
    def _hash_bytes(*parts: bytes) -> bytes:
        h = hashlib.sha256()
        for p in parts: h.update(p)
        return h.digest()

    def _epoch_seed_str(self) -> str:
        return f"takamaka-seed:{self.cfg['seed']}:{self.epoch}"

    # ---------------------------
    # Pesi effettivi (stake attenuata da penalty)
    # ---------------------------
    def _compute_effective_weights_now(self) -> Dict[str, float]:
        weights: Dict[str, float] = {}
        alpha = self.cfg["penalty_alpha"]
        for node in self.overflows.values():
            main = self.mains[node.main_id]
            base = max(0.0, main.active_stake * max(0.0, node.base_stake_share))
            resid = getattr(node, "penalty_slots_residual", 0)
            eff = base / (1.0 + alpha * max(0, int(resid)))
            node.penalty_score = alpha * max(0, int(resid))
            weights[node.address] = eff
        return weights

    @staticmethod
    def _int_addr(addr: str) -> int:
        return int.from_bytes(hashlib.sha256(addr.encode()).digest(), "big", signed=False)

    def _build_intervals_from_weights_vrf(self, weights: Dict[str, float]) -> Tuple[List[str], List[int], int]:
        scale = int(self.cfg["stake_scale_int"])
        items = []
        for addr, w in weights.items():
            wint = int((0.0 if w is None else float(w)) * scale)
            if wint <= 0: continue
            items.append((addr, self._int_addr(addr), wint))
        items.sort(key=lambda x: x[1])

        addresses, prefix_ends, acc = [], [], 0
        for addr, _, wint in items:
            addresses.append(addr); acc += wint; prefix_ends.append(acc)
        total_int = acc
        return addresses, prefix_ends, total_int

    @staticmethod
    def _iter_hash(seed: bytes, n: int) -> bytes:
        x = seed
        for _ in range(n): x = hashlib.sha256(x).digest()
        return x

    def _vrf_schedule_exact(self, seed: bytes, weights: Dict[str, float]) -> Tuple[Dict[str, int], List[str]]:
        S = self.cfg["slots_per_epoch"]
        addrs, prefix_ends, total_int = self._build_intervals_from_weights_vrf(weights)
        if total_int <= 0 or not addrs:
            return {addr: 0 for addr in self.overflows}, []
        counts = {addr: 0 for addr in self.overflows}
        schedule: List[str] = []
        for i in range(S):
            ri = self._iter_hash(seed, i + 1)
            ticket = int.from_bytes(ri, "big") % total_int
            j = bisect.bisect_left(prefix_ends, ticket)
            if j >= len(addrs): j = len(addrs) - 1
            addr = addrs[j]
            schedule.append(addr)
            counts[addr] = counts.get(addr, 0) + 1
        return counts, schedule

    # ---------------------------
    # Penalità (solo qui) — reward si accreditano per slot OK
    # ---------------------------
    def _apply_penalties(self, per_node_outcomes: Dict[str, Dict[str, int]]):
        wS = self.cfg["w_skipped"]; wI = self.cfg["w_invalid"]; wD = self.cfg["w_delayed"]
        PF = self.cfg["penalty_factor"]; RF = self.cfg["recovery_factor"]; tol_ratio = self.cfg["tolerated_bad_ratio"]

        for addr, outcome in per_node_outcomes.items():
            node = self.overflows[addr]
            assigned = max(1, node.assigned_slots)
            ok      = int(outcome.get("ok", 0))
            skipped = int(outcome.get("skipped", 0))
            invalid = int(outcome.get("invalid", 0))
            delayed = int(outcome.get("delayed", 0))

            tol = int(tol_ratio * assigned)
            bad_weighted = (wS * skipped) + (wI * invalid) + (wD * delayed)
            gained = max(0.0, bad_weighted - tol) * PF
            gained_i = int(math.ceil(gained))

            resid_prev = getattr(node, "penalty_slots_residual", 0)
            resid_next = max(0, int(resid_prev) - int(RF * ok)) + gained_i
            node.penalty_slots_residual = int(resid_next)

            node.penalty_score = self.cfg["penalty_alpha"] * node.penalty_slots_residual

            spiv = self.cfg["slash_per_invalid"]
            if spiv > 0.0 and invalid > 0:
                node.slash_green(invalid * spiv)

    # ---------------------------
    # Mempool & blocco
    # ---------------------------
    def _wallet_by_addr(self, addr: str) -> Optional[Wallet]:
        return self.stakeholders.get(addr) or self.mains.get(addr) or self.overflows.get(addr)

    def _gen_tx_batch(self, rng, n: int) -> List[Transaction]:
        if n <= 0: return []
        parties = list(self.stakeholders.keys()) + list(self.mains.keys()) + list(self.overflows.keys())
        if len(parties) < 2: return []
        kinds = list(FEE_SCHEDULE_TKR.keys())
        txs: List[Transaction] = []
        for i in range(n):
            s = rng.choice(parties); t = rng.choice(parties)
            while t == s: t = rng.choice(parties)
            k = rng.choices(kinds, weights=[70,6,4,1,2,1,1,2,1,12], k=1)[0]  # più PAY e CALL_FUNCTION
            amt_g = rng.random()*5.0 if k=="PAY" and rng.random()<0.3 else 0.0
            amt_r = rng.random()*5.0 if k=="PAY" and rng.random()<0.7 else 0.0
            txs.append(Transaction(
                txid=hashlib.sha256(f"{self.epoch}|{i}|{s}|{t}|{k}".encode()).hexdigest()[:16],
                kind=k, sender=s, recipient=t, amount_green=amt_g, amount_red=amt_r, valid=True
            ))
        return txs

    def _calc_required_fee_tkr(self, tx: Transaction) -> float:
        return FEE_SCHEDULE_TKR.get(tx.kind, 0.0)

    def _pay_fee_and_apply_payload(self, tx: Transaction) -> bool:
        """Paga la fee (TKR→TKG fallback) e applica un payload minimale (trasferimenti PAY)."""
        src = self._wallet_by_addr(tx.sender); dst = self._wallet_by_addr(tx.recipient)
        if not src or not dst: return False

        # verifica fee pagata
        required = self._calc_required_fee_tkr(tx)
        paid = tx.fee_paid_tkr
        if paid + 1e-18 < required:  # sottopagata
            return False

        # incassa fee dal sender: prima TKR poi TKG (fallback)
        need = required
        take_r = min(src.red, need); src.red -= take_r; need -= take_r
        if need > 0:
            take_g = min(src.green, need); src.green -= take_g; need -= take_g
        if need > 0:  # non riesce a pagare davvero
            return False

        # payload (solo PAY, gli altri tipi non cambiano stato in questa versione)
        if tx.kind == "PAY":
            if src.green < tx.amount_green or src.red < tx.amount_red:
                return False
            if tx.amount_green > 0: src.green -= tx.amount_green; dst.green += tx.amount_green
            if tx.amount_red   > 0: src.red   -= tx.amount_red;   dst.red   += tx.amount_red

        return True

    # ---------------------------
    # Seed next
    # ---------------------------
    def _compute_next_seed_from_eep_block(self) -> bytes:
        eep_idx = max(0, self.cfg["eep_slot"] - 1)
        chosen_slot = None
        if self._slot_outcomes_current:
            i = min(eep_idx, len(self._slot_outcomes_current) - 1)
            while i >= 0:
                if self._slot_outcomes_current[i] == "ok":
                    chosen_slot = i; break
                i -= 1
        if chosen_slot is None:
            return hashlib.sha256(b"fallback|seed|" + str(self.epoch).encode()).digest()
        producer_addr = self.current_schedule[chosen_slot]
        block_message = (b"|epoch|" + self.epoch.to_bytes(8, "big") +
                         b"|slot|" + chosen_slot.to_bytes(8, "big") +
                         b"|producer|" + producer_addr.encode())
        return hashlib.sha256(block_message).digest()

    def _prepare_next_epoch_state(self):
        self._apply_ops_at_current_eep()
        self._snapshot_all_mains_at_eep()
        self.weights_next_epoch = self._compute_effective_weights_now()
        self.next_seed = self._compute_next_seed_from_eep_block()
        _, self.next_schedule = self._vrf_schedule_exact(self.next_seed, self.weights_next_epoch)

    def _bootstrap_epoch0_if_needed(self):
        if self.epoch != 0 or self._bootstrap_done: return
        # Applica solo le op (epoch 0, slot 0)
        eligible, keep = [], []
        for op in self.pending_ops:
            if op["epoch"] == 0 and op["slot"] == 0: eligible.append(op)
            else: keep.append(op)
        eligible.sort(key=lambda op: (0 if op["kind"] in ("STAKE_UNDO","SELF_UNBET") else 1,
                                      op.get("epoch",-1), op.get("slot",-1), op.get("from",""), op.get("to","")))
        for op in eligible:
            kind = op["kind"]
            if kind == "STAKE_UNDO":
                deleg_addr = op["from"]; st = self.stakeholders.get(deleg_addr)
                if st:
                    for m in self.mains.values():
                        cur = m.delegates.get(deleg_addr, 0.0)
                        if cur > 0: st.green += cur; m.delegates.pop(deleg_addr, None)
            elif kind == "STAKE":
                to_main = self.mains.get(op["to"])
                if to_main:
                    amount = float(op["amount"])
                    if amount >= MIN_BET_AMOUNT:
                        from_addr = op["from"]
                        if from_addr == "_self_":
                            if amount <= to_main.green:
                                to_main.green -= amount; to_main.bet("_self_", amount)
                        else:
                            st = self.stakeholders.get(from_addr)
                            if st and amount <= st.green:
                                st.green -= amount; to_main.bet(from_addr, amount)
            elif kind == "SELF_UNBET":
                to_main = self.mains.get(op["to"])
                if to_main: to_main.self_unbet(op.get("amount", None))
        self.pending_ops = keep

        self._snapshot_all_mains_at_eep()
        self.weights_for_epoch = self._compute_effective_weights_now()
        self.current_seed = hashlib.sha256(b"bootstrap-seed|" + str(self.cfg['seed']).encode()).digest()
        _, self.current_schedule = self._vrf_schedule_exact(self.current_seed, self.weights_for_epoch)
        self.next_seed = self.current_seed
        self.next_schedule = list(self.current_schedule)
        self._initialized_weights = True
        self._bootstrap_done = True

    # ---------------------------
    # Simulazione di un'epoca
    # ---------------------------
    def simulate_one_epoch(self) -> Dict:
        if self.epoch == 0 and not self._bootstrap_done:
            self._bootstrap_epoch0_if_needed()
        elif not self.current_schedule:
            if not self._initialized_weights:
                self.weights_for_epoch = self._compute_effective_weights_now()
                self._initialized_weights = True
            self.current_seed = hashlib.sha256(self._epoch_seed_str().encode() + b"|current-seed|").digest()
            _, self.current_schedule = self._vrf_schedule_exact(self.current_seed, self.weights_for_epoch)

        # aggiorna stake effettiva
        for addr, node in self.overflows.items():
            node.effective_stake = float(self.weights_for_epoch.get(addr, 0.0))

        per_node_outcomes = _dd(lambda: {"ok": 0, "skipped": 0, "invalid": 0, "delayed": 0})
        fees_epoch_by_main = _dd(float)  # TKR raccolte per main
        self._slot_outcomes_current = []

        rng_slots = _rnd.Random(self.cfg["seed"] * 7919 + self.epoch)

        # mempool: genera tx per tutti gli slot (media txs_per_slot_mean)
        self.mempool = []
        for s in range(self.cfg["slots_per_epoch"]):
            self.mempool.extend(self._gen_tx_batch(rng_slots, self.cfg["txs_per_slot_mean"]))

        # esegui slot
        slots_per_node = {addr: 0 for addr in self.overflows}
        for slot_idx, addr in enumerate(self.current_schedule):
            node = self.overflows[addr]
            main = self.mains[node.main_id]
            slots_per_node[addr] = slots_per_node.get(addr, 0) + 1

            # esito "di rete" (skipped/invalid/delayed/ok)
            outcome = node.behavior_outcome(rng_slots)

            # costruiamo un blocco solo se non è skipped
            block_valid = (outcome == "ok")
            collected_fees_tkr = 0.0

            if outcome != "skipped":
                # selezione tx dal mempool
                picked = self.mempool[:self.cfg["max_txs_per_block"]]
                self.mempool = self.mempool[self.cfg["max_txs_per_block"]:]
                had_invalid_tx = False

                # “malevolo”: con p_bad_tx marca alcune tx come invalide o sottopagate
                malicious = (rng_slots.random() < node.p_bad_tx)

                for tx in picked:
                    # decide fee pagata (onesto = required, malevolo può sottopagare ~50%)
                    required = self._calc_required_fee_tkr(tx)
                    if malicious and rng_slots.random() < 0.5:
                        tx.valid = False  # payload/firma corrotta
                    tx.fee_paid_tkr = required if not (malicious and rng_slots.random() < 0.5) else required * 0.5

                    # valida & applica (paga fee + payload PAY)
                    ok_tx = tx.valid and self._pay_fee_and_apply_payload(tx)
                    if not ok_tx:
                        had_invalid_tx = True
                        break
                    collected_fees_tkr += required  # le fee incassate “di protocollo” sono le required

                if had_invalid_tx:
                    block_valid = False

            # contabilizzazione slot
            if block_valid and outcome == "ok":
                per_node_outcomes[addr]["ok"] += 1
                self._slot_outcomes_current.append("ok")
                # reward: coinbase in TKG + fees in TKR
                main.credit_reward(self.cfg["reward_per_block"])  # TKG
                main.deposit(red=collected_fees_tkr)             # TKR
                fees_epoch_by_main[main.address] += collected_fees_tkr
            else:
                # se outcome già invalid/delayed/… li rispettiamo
                if outcome in ("invalid", "delayed"): per_node_outcomes[addr][outcome] += 1
                elif outcome == "skipped":            per_node_outcomes[addr]["skipped"] += 1
                else:                                 per_node_outcomes[addr]["invalid"] += 1  # invalid per tx errate
                self._slot_outcomes_current.append("invalid" if outcome=="ok" else outcome)

        # salva assigned_slots per penalty
        for addr, node in self.overflows.items():
            node.assigned_slots = slots_per_node.get(addr, 0)

        # applica penalità (reward già accreditate slot-per-slot)
        self._apply_penalties(per_node_outcomes)

        # prepara prossimo epoch
        self._prepare_next_epoch_state()

        total_effective_stake = sum(self.weights_for_epoch.values())
        total_slots_epoch     = int(self.cfg["slots_per_epoch"])

        kpi = {
            "epoch": self.epoch,
            "slots_per_node": dict(slots_per_node),
            "per_node_outcomes": {addr: dict(v) for addr, v in per_node_outcomes.items()},
            "penalty_slots_residual": {addr: getattr(self.overflows[addr], "penalty_slots_residual", 0)
                                       for addr in self.overflows},
            "penalties_alpha_scaled": {addr: self.overflows[addr].penalty_score for addr in self.overflows},
            "effective_stake_per_overflow": {addr: self.overflows[addr].effective_stake for addr in self.overflows},
            "total_effective_stake": total_effective_stake,
            "total_slots": total_slots_epoch,
            "fees_epoch_by_main": dict(fees_epoch_by_main),  # TKR per main in questo epoch
            "seed_current_hex": self.current_seed.hex(),
            "seed_next_hex": self.next_seed.hex(),
        }
        self.history.append(kpi)

        # rollover
        self.weights_for_epoch = dict(self.weights_next_epoch)
        self.current_seed = self.next_seed
        self.current_schedule = list(self.next_schedule)
        self.epoch += 1
        return kpi

    # ---------------------------
    # Report epoch → DataFrame
    # ---------------------------
    def epoch_summary_df(self, kpi: Dict) -> pd.DataFrame:
        total_effective = kpi.get("total_effective_stake", 0.0)
        total_slots_epoch = int(kpi.get("total_slots", 0))
        reward_per_block = float(self.cfg.get("reward_per_block", 0.0))

        # per ricostruire reward cumulate coinbase+fee:
        # coinbase cumulata per overflow = (#ok cumulati) * reward_per_block
        ok_cum_map: Dict[str, int] = _dd(int)
        fees_cum_by_main: Dict[str, float] = _dd(float)

        for past in self.history:
            for addr, o in past.get("per_node_outcomes", {}).items():
                ok_cum_map[addr] += int(o.get("ok", 0))
            for m, f in past.get("fees_epoch_by_main", {}).items():
                fees_cum_by_main[m] += float(f)

        rows = []
        for addr, slots in kpi["slots_per_node"].items():
            o = kpi["per_node_outcomes"].get(addr, {})
            node = self.overflows[addr]
            main = self.mains[node.main_id]

            ok = int(o.get("ok", 0))
            skipped = int(o.get("skipped", 0))
            invalid = int(o.get("invalid", 0))
            delayed = int(o.get("delayed", 0))
            bad_abs = skipped + invalid + delayed

            stake_share_pct = (node.effective_stake / total_effective * 100.0) if total_effective > 0 else 0.0
            delivered_slots_share_pct = (ok / total_slots_epoch * 100.0) if total_slots_epoch > 0 else 0.0
            bad_slots_pct_node = (bad_abs / slots * 100.0) if slots > 0 else 0.0

            # reward epoch (per overflow): coinbase TKG + fee TKR del suo main per i blocchi OK
            coinbase_epoch = ok * reward_per_block
            fees_epoch    = float(kpi.get("fees_epoch_by_main", {}).get(main.address, 0.0))  # totale main (sommato su suoi overflow)
            # Per attribuire “per overflow” le fee del main potresti ripartirle pro-rata sugli OK dell'overflow.
            # Qui mostriamo le fee a livello main (colonna separata) e la coinbase per overflow.
            reward_epoch_total_value = coinbase_epoch  # (in TKG). Le fee sono in TKR: le riportiamo a parte.

            # cumulati
            coinbase_cum = ok_cum_map[addr] * reward_per_block
            fees_cum_main = fees_cum_by_main[main.address]

            rows.append({
                "epoch": kpi["epoch"],
                "overflow": addr,
                "main": main.address,
                "assigned_slots": int(slots),

                "ok": ok, "skipped": skipped, "invalid": invalid, "delayed": delayed,
                "bad_slots_abs": int(bad_abs), "bad_slots_pct_node": round(bad_slots_pct_node, 2),

                "effective_stake": node.effective_stake,
                "stake_share_pct": round(stake_share_pct, 2),
                "delivered_slots_share_pct": round(delivered_slots_share_pct, 2),

                "coinbase_epoch_TKG": round(coinbase_epoch, 6),
                "fees_epoch_TKR_at_main": round(fees_epoch, 6),
                "reward_epoch_total_value_TKG_only": round(reward_epoch_total_value, 6),

                "coinbase_cum_TKG": round(coinbase_cum, 6),
                "fees_cum_TKR_at_main": round(fees_cum_main, 6),

                "penalty_slots_residual": getattr(node, "penalty_slots_residual", 0),
                "penalty_score": node.penalty_score,
            })

        return pd.DataFrame(rows)

# Simulations

## Simulation 1

This simulation runs for 20 epochs.
- All nodes begin with stake of 100
- All nodes behave correctly, meaning that every assigned slot is validated as “ok.”

The final report is saved to ./data/simulation_1.csv.

In [12]:
sim = TakamakaSim(
    slots_per_epoch=SLOTS_PER_EPOCH,
    slot_duration_sec=SLOT_DURATION_SEC,
    seed=RNG_SEED
)

mA = MainAddress(address="MAIN_A", green=0.0)
mB = MainAddress(address="MAIN_B", green=0.0)
mC = MainAddress(address="MAIN_C", green=0.0)

sim.register_main(mA)
sim.register_main(mB)
sim.register_main(mC)

oA = OverflowNode(address="OF_A", green=0.0, p_skip=0.0, p_invalid=0.0, p_delayed=0.0)
oB = OverflowNode(address="OF_B", green=0.0, p_skip=0.0, p_invalid=0.0, p_delayed=0.0)
oC = OverflowNode(address="OF_C", green=0.0, p_skip=0.0, p_invalid=0.0, p_delayed=0.0)

mA.register_overflow(oA); sim.index_overflow(oA)
mB.register_overflow(oB); sim.index_overflow(oB)
mC.register_overflow(oC); sim.index_overflow(oC)

sA = Stakeholder(address="STK_A", green=100.0)
sB = Stakeholder(address="STK_B", green=100.0)
sC = Stakeholder(address="STK_C", green=100.0)

sim.register_stakeholder(sA)
sim.register_stakeholder(sB)
sim.register_stakeholder(sC)

sim.queue_bet(from_addr="STK_A", to_main_id="MAIN_A", amount=100.0, when_slot=0)
sim.queue_bet(from_addr="STK_B", to_main_id="MAIN_B", amount=100.0, when_slot=0)
sim.queue_bet(from_addr="STK_C", to_main_id="MAIN_C", amount=100.0, when_slot=0)

kpis = []
dfs = []

for _ in range(20):
    kpi = sim.simulate_one_epoch()
    kpis.append(kpi)
    dfs.append(sim.epoch_summary_df(kpi))

summary = pd.concat(dfs, ignore_index=True)
out_path = "./data/simulation_ok.csv"
summary.to_csv(out_path, index=False)


## Simulation 2

- Identical setup to Simulation 1.
- The only change is that Node 1 behaves maliciously with error probabilities p_skip = 0.02 and p_invalid = 0.01 (others remain honest). 

The final report is saved to ./data/simulation_2.csv.

In [10]:
sim = TakamakaSim(
    slots_per_epoch=SLOTS_PER_EPOCH,
    slot_duration_sec=SLOT_DURATION_SEC,
    seed=RNG_SEED
)

mA = MainAddress(address="MAIN_A", green=0.0)
mB = MainAddress(address="MAIN_B", green=0.0)
mC = MainAddress(address="MAIN_C", green=0.0)
sim.register_main(mA); sim.register_main(mB); sim.register_main(mC)

oA = OverflowNode(address="OF_A", green=0.0, p_skip=0.02, p_invalid=0.01, p_delayed=0.0)
oB = OverflowNode(address="OF_B", green=0.0, p_skip=0.0,  p_invalid=0.0,  p_delayed=0.0)
oC = OverflowNode(address="OF_C", green=0.0, p_skip=0.0,  p_invalid=0.0,  p_delayed=0.0)
mA.register_overflow(oA); sim.index_overflow(oA)
mB.register_overflow(oB); sim.index_overflow(oB)
mC.register_overflow(oC); sim.index_overflow(oC)

sA = Stakeholder(address="STK_A", green=100.0)
sB = Stakeholder(address="STK_B", green=100.0)
sC = Stakeholder(address="STK_C", green=100.0)
sim.register_stakeholder(sA); sim.register_stakeholder(sB); sim.register_stakeholder(sC)

sim.queue_bet(from_addr="STK_A", to_main_id="MAIN_A", amount=100.0, when_slot=0)
sim.queue_bet(from_addr="STK_B", to_main_id="MAIN_B", amount=100.0, when_slot=0)
sim.queue_bet(from_addr="STK_C", to_main_id="MAIN_C", amount=100.0, when_slot=0)

dfs = []
for _ in range(20):
    kpi = sim.simulate_one_epoch()
    dfs.append(sim.epoch_summary_df(kpi))

summary_faulty = pd.concat(dfs, ignore_index=True)

out_path = "data/simulation_faulty.csv"
summary_faulty.to_csv(out_path, index=False)